In [2]:
import numpy as np
import pandas as pd
import requests

In [3]:
#weather_dir = 'I:/Weather Data/'
weather_dir = '.'

In [10]:
daily_tmin_df = pd.read_fwf(f'{weather_dir}/dly-tmin-normal.txt', header=None)
daily_tmin_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmin_df = pd.melt(daily_tmin_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmin_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmin_df = daily_tmin_df[daily_tmin_df['VALUE'] != '-8888']
daily_tmin_df['FLAG'] = daily_tmin_df['VALUE'].map(lambda x: x[-1])
daily_tmin_df['VALUE'] = daily_tmin_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmin_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,77.2,C
1,AQW00061705,2,1,77.4,C
2,AQW00061705,3,1,77.6,C
3,AQW00061705,4,1,77.4,C
4,AQW00061705,5,1,77.3,C
5,AQW00061705,6,1,77.2,C
6,AQW00061705,7,1,76.5,C
7,AQW00061705,8,1,76.2,C
8,AQW00061705,9,1,76.5,C
9,AQW00061705,10,1,76.8,C


In [11]:
daily_tmax_df = pd.read_fwf(f'{weather_dir}/dly-tmax-normal.txt', header=None)
daily_tmax_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmax_df = pd.melt(daily_tmax_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmax_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmax_df = daily_tmax_df[daily_tmax_df['VALUE'] != '-8888']
daily_tmax_df['FLAG'] = daily_tmax_df['VALUE'].map(lambda x: x[-1])
daily_tmax_df['VALUE'] = daily_tmax_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmax_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,87.5,C
1,AQW00061705,2,1,87.8,C
2,AQW00061705,3,1,88.2,C
3,AQW00061705,4,1,87.9,C
4,AQW00061705,5,1,86.9,C
5,AQW00061705,6,1,85.5,C
6,AQW00061705,7,1,84.6,C
7,AQW00061705,8,1,84.0,C
8,AQW00061705,9,1,84.9,C
9,AQW00061705,10,1,85.6,C


In [12]:
daily_temp_df = pd.merge(daily_tmin_df, daily_tmax_df, on=['STNID','MONTH','DAY'], suffixes=['_MIN','_MAX'])
daily_temp_df

,STNID,MONTH,DAY,VALUE_MIN,FLAG_MIN,VALUE_MAX,FLAG_MAX
0,AQW00061705,1,1,77.2,C,87.5,C
1,AQW00061705,2,1,77.4,C,87.8,C
2,AQW00061705,3,1,77.6,C,88.2,C
3,AQW00061705,4,1,77.4,C,87.9,C
4,AQW00061705,5,1,77.3,C,86.9,C
5,AQW00061705,6,1,77.2,C,85.5,C
6,AQW00061705,7,1,76.5,C,84.6,C
7,AQW00061705,8,1,76.2,C,84.0,C
8,AQW00061705,9,1,76.5,C,84.9,C
9,AQW00061705,10,1,76.8,C,85.6,C


In [13]:
all_stations_df = pd.read_fwf(f'{weather_dir}/allstations.txt', header=None)
all_stations_df.columns = ['ID', 'Latitude','Longitude','Elevation','State','Name','GSNFLAG','HCNFLAG','WMOID']
all_stations_df.head()

,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID
0,AQC00914000,-14.3167,-170.7667,408.4,AS,AASUFOU,NaN,NaN,NaN
1,AQW00061705,-14.3306,-170.7136,3.7,AS,PAGO PAGO WSO AP,NaN,GSN,91765.0
2,CAW00064757,44.2325,-79.7811,246.0,ON,EGBERT 1 W,NaN,NaN,NaN
3,CQC00914080,15.2136,145.7497,252.1,MP,CAPITOL HILL 1,NaN,NaN,NaN
4,CQC00914801,14.1717,145.2428,179.2,MP,ROTA AP,NaN,NaN,91221.0


In [17]:
# lat_longs = list(stations.apply(lambda row: {'Latitude':row['Latitude'],'Longitude':row['Longitude']}, axis='columns'))
# lat_longs

# for lat_long in lat_longs:
#     url = f"https://geo.fcc.gov/api/census/block/find?latitude={lat_long['Latitude']}&longitude={lat_long['Longitude']}&showall=false&format=json"
#     response = requests.get(url)
#     if response.ok:
#         lat_long['FIPS_COUNTY'] = response.json()['County']['FIPS']
#     print(lat_long)

# pd.DataFrame(lat_longs).to_csv('lat_longs_to_fips.csv')

lat_longs = pd.read_csv('lat_longs_to_fips.csv', dtype={'FIPS_COUNTY':str})
lat_longs

,Latitude,Longitude,FIPS_COUNTY
0,34.2553,-87.1814,01133
1,32.9453,-85.9481,01123
2,33.1272,-88.1550,01107
3,31.3072,-86.5225,01039
4,34.9092,-87.2747,01077
5,33.2942,-85.7789,01027
6,34.7753,-86.9508,01083
7,31.1819,-87.4389,01053
8,32.5992,-85.4653,01081
9,32.6000,-85.5000,01081


In [24]:
daily_temp_df = pd.merge(daily_temp_df, all_stations_df, left_on='STNID', right_on='ID')
daily_temp_df = pd.merge(daily_temp_df, lat_longs, on=['Latitude','Longitude'])
daily_temp_df

,STNID,MONTH,DAY,VALUE_MIN,FLAG_MIN,VALUE_MAX,FLAG_MAX,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID,FIPS_COUNTY
0,USC00010063,1,1,29.1,Q,49.6,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
1,USC00010063,2,1,30.3,Q,51.8,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
2,USC00010063,3,1,36.1,Q,58.1,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
3,USC00010063,4,1,44.0,Q,68.0,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
4,USC00010063,5,1,52.1,Q,74.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
5,USC00010063,6,1,61.0,Q,83.1,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
6,USC00010063,7,1,67.1,Q,87.2,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
7,USC00010063,8,1,68.2,Q,89.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
8,USC00010063,9,1,65.0,Q,87.6,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
9,USC00010063,10,1,54.0,Q,77.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133


In [51]:
daily_min_temp_by_county = daily_temp_df.groupby(['FIPS_COUNTY','MONTH','DAY'])['VALUE_MIN'].min()
daily_max_temp_by_county = daily_temp_df.groupby(['FIPS_COUNTY','MONTH','DAY'])['VALUE_MAX'].max()
lat_long_by_county = daily_temp_df.groupby(['FIPS_COUNTY'])[['Latitude','Longitude']].mean()
daily_temp_by_county = pd.merge(min_temp_by_county, max_temp_by_county, left_index=True, right_index=True)
daily_temp_by_county = pd.merge(daily_temp_by_county, lat_long_by_county, left_index=True, right_index=True)
daily_temp_by_county

VALUE_MIN  VALUE_MAX   Latitude  Longitude
FIPS_COUNTY MONTH DAY                                            
01003       1     1         37.6       61.3  30.636125   -87.8109
                  2         37.5       61.3  30.636125   -87.8109
                  3         37.4       61.2  30.636125   -87.8109
                  4         37.3       61.2  30.636125   -87.8109
                  5         37.2       61.2  30.636125   -87.8109
...                          ...        ...        ...        ...
56045       12    27         7.4       35.1  43.927500  -104.5146
                  28         7.4       35.2  43.927500  -104.5146
                  29         7.4       35.3  43.927500  -104.5146
                  30         7.5       35.4  43.927500  -104.5146
                  31         7.5       35.5  43.927500  -104.5146

[1049322 rows x 4 columns]

In [75]:
precip_df = pd.read_fwf(f'{weather_dir}/ann-prcp-normal.txt', header=None)
precip_df.columns = ['STNID','VALUE']
precip_df['FLAG'] = precip_df['VALUE'].map(lambda x: x[-1])
precip_df['annual_precipitation'] = precip_df['VALUE'].map(lambda x: int(x[0:-1])/100)

precip_df = pd.merge(precip_df, all_stations_df, left_on='STNID', right_on='ID')
precip_df = pd.merge(precip_df, lat_longs, on=['Latitude','Longitude'])

precip_by_county = precip_df.groupby('FIPS_COUNTY')['annual_precipitation'].max()
#precip_by_county['13193']
#precip_df[precip_df['FIPS_COUNTY'] == '13193']
precip_df[precip_df['FIPS_COUNTY'] == '01003']

,STNID,VALUE,FLAG,annual_precipitation,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID,FIPS_COUNTY
11,USC00010583,6890S,S,68.90,USC00010583,30.8839,-87.7853,82.6,AL,BAY MINETTE,NaN,NaN,NaN,01003
28,USC00012813,6787S,S,67.87,USC00012813,30.5467,-87.8808,7.0,AL,FAIRHOPE 2 NE,NaN,NaN,NaN,01003
60,USC00016988,6732R,R,67.32,USC00016988,30.5653,-87.7017,49.1,AL,ROBERTSDALE,NaN,NaN,NaN,01003


In [69]:
df = pd.DataFrame({
    'min_temp': daily_temp_by_county.groupby('FIPS_COUNTY')['VALUE_MIN'].min(), 
    'max_temp':daily_temp_by_county.groupby('FIPS_COUNTY')['VALUE_MAX'].max(),
    'latitude':daily_temp_by_county.groupby('FIPS_COUNTY')['Latitude'].mean(),
    'longitude':daily_temp_by_county.groupby('FIPS_COUNTY')['Longitude'].mean(),
    'annual_precipitation':precip_by_county
}).reset_index()
df.rename({'index':'FIPS_COUNTY'}, axis='columns', inplace=True)
df['annual_precipitation'].isna().sum()

46

In [53]:
avg_weekly_wage = pd.read_csv('avg_weekly_wage.csv', dtype={'FIPS_COUNTY':str})
avg_weekly_wage.head()

,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01001,"Autauga County, Alabama",802
1,01003,"Baldwin County, Alabama",717
2,01005,"Barbour County, Alabama",727
3,01007,"Bibb County, Alabama",785
4,01009,"Blount County, Alabama",683


In [54]:
data_df = pd.merge(df, avg_weekly_wage, on='FIPS_COUNTY')
data_df

,FIPS_COUNTY,min_temp,max_temp,latitude,longitude,annual_precipitation,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01003,36.8,91.2,30.636125,-87.810900,68.90,"Baldwin County, Alabama",717
1,01005,31.8,93.4,31.944433,-85.222967,50.14,"Barbour County, Alabama",727
2,01007,30.3,90.9,32.866100,-87.238300,57.23,"Bibb County, Alabama",785
3,01009,29.9,90.8,33.947800,-86.469200,55.41,"Blount County, Alabama",683
4,01011,32.9,90.6,32.014200,-85.746400,55.42,"Bullock County, Alabama",673
5,01013,33.1,90.6,31.790000,-86.608600,57.36,"Butler County, Alabama",659
6,01015,32.7,90.1,33.587200,-85.855600,49.67,"Calhoun County, Alabama",794
7,01017,29.1,90.9,32.888150,-85.311400,54.55,"Chambers County, Alabama",745
8,01019,27.7,90.5,34.150000,-85.684700,54.10,"Cherokee County, Alabama",658
9,01021,31.9,91.0,32.864200,-86.644800,57.88,"Chilton County, Alabama",741


In [79]:
data_df['annual_precipitation'] = data_df['annual_precipitation'].fillna(-1)
data_df.to_csv('data.csv')

In [59]:
data_df.head()

,FIPS_COUNTY,min_temp,max_temp,latitude,longitude,annual_precipitation,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01003,36.8,91.2,30.636125,-87.810900,68.90,"Baldwin County, Alabama",717
1,01005,31.8,93.4,31.944433,-85.222967,50.14,"Barbour County, Alabama",727
2,01007,30.3,90.9,32.866100,-87.238300,57.23,"Bibb County, Alabama",785
3,01009,29.9,90.8,33.947800,-86.469200,55.41,"Blount County, Alabama",683
4,01011,32.9,90.6,32.014200,-85.746400,55.42,"Bullock County, Alabama",673
